In [3]:
# unified_agent.py

import os
from dotenv import load_dotenv

from langchain.agents import initialize_agent, Tool, AgentType
from langchain_openai import ChatOpenAI

# --- PDF (RAG) pieces
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

# --- SQL pieces
from langchain_community.utilities import SQLDatabase
from langchain.agents.agent_toolkits import create_sql_agent


# -------------------------------
# 0) Config / Keys
# -------------------------------
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

DB_PATH = "/Users/sanket/Documents/Python Projects/New Project LLM MultiDB/healthcare.db"
FAISS_PATH = "faiss_index_notice_privacy"  # folder containing index.faiss + index.pkl

# -------------------------------
# 1) Shared LLM
# -------------------------------
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# -------------------------------
# 2) PDF Tool (RAG over FAISS)
# -------------------------------
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(
    FAISS_PATH,
    embeddings=embedding,
    allow_dangerous_deserialization=True,  # needed to load index.pkl
)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

pdf_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,   # so we can show which PDF & page
    chain_type="stuff",
)

def run_pdf_tool(question: str) -> str:
    result = pdf_qa.invoke({"query": question})
    sources = result.get("source_documents", [])
    if sources:
        unique_srcs = []
        for doc in sources:
            src = f"{doc.metadata.get('source','PDF')} (p.{doc.metadata.get('page','?')})"
            if src not in unique_srcs:
                unique_srcs.append(src)
        src_line = ", ".join(unique_srcs[:5])
    else:
        src_line = "PDF index (no page metadata)"

    return (
        f"Source: PDF • Files: {src_line}\n"
        f"Tool Used: PDF_RetrievalQA\n"
        f"Answer: {result['result']}"
    )

pdf_tool = Tool(
    name="PDF_RetrievalQA",
    func=run_pdf_tool,
    description=(
        "Use for questions about content in the PDFs (privacy policy, patient rights, website policy, "
        "pharmacy coverage, user guide, etc.). INPUT MUST BE THE ORIGINAL NATURAL-LANGUAGE QUESTION."
    ),
)

# -------------------------------
# 3) SQL Tool (Healthcare DB) — robust wrapper
# -------------------------------
db = SQLDatabase.from_uri(f"sqlite:///{DB_PATH}")

sql_agent = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    verbose=False,
    prefix="""
You are a helpful medical data assistant.

Database schema:
- patients(patient_id PK, name, age, gender)
- visits(visit_id PK, patient_id FK->patients.patient_id, date, reason)
- prescriptions(id PK, visit_id FK->visits.visit_id, med_id FK->medications.med_id, dosage)
- medications(med_id PK, name, category)

Rules:
- Conditions (e.g., 'hypertension', 'chest pain') live in visits.reason (string match; use LOWER() when needed).
- For patient 'summary', join patients -> visits -> prescriptions -> medications and order by date.
- Prefer DISTINCT to avoid duplicates where it makes sense.
- Return concise, faithful results. Do not invent data.

When summarizing, output a short clinical-style paragraph (no bullets).
"""
)

SQL_PREFIX_FORCED = """You MUST accept a NATURAL LANGUAGE question and generate SQL yourself.
Do NOT expect the input to be SQL. If the input looks like SQL anyway, execute it directly and then summarize the results."""

def run_sql_tool(question: str) -> str:
    q = question.strip()
    looks_like_sql = q.lower().startswith(("select", "with", "pragma", "explain", "insert", "update", "delete"))
    try:
        if looks_like_sql:
            # Execute raw SQL if router mistakenly sends SQL
            rows = db.run(q)
            return (
                "Source: Database (SQLite: healthcare.db)\n"
                "Tool Used: SQL_Agent (direct SQL execution)\n"
                f"Answer: {rows}"
            )
        # Normal path: let the SQL agent generate/explain SQL from NL
        answer = sql_agent.run(SQL_PREFIX_FORCED + "\n\nUser question:\n" + q)
        return (
            "Source: Database (SQLite: healthcare.db)\n"
            "Tool Used: SQL_Agent\n"
            f"Answer: {answer}"
        )
    except Exception as e:
        return (
            "Source: Database (SQLite: healthcare.db)\n"
            "Tool Used: SQL_Agent\n"
            f"Answer: Error while querying database: {e}"
        )

sql_tool = Tool(
    name="SQL_Agent",
    func=run_sql_tool,
    description=(
        "Use for questions about patients, visits, prescriptions, medications, counts or summaries in the SQLite database. "
        "INPUT MUST BE THE ORIGINAL NATURAL-LANGUAGE QUESTION. Do NOT pass SQL; the tool will generate SQL."
    ),
)

# -------------------------------
# 4) Router Agent (auto-selects tool) — strict routing instructions
# -------------------------------
tools = [pdf_tool, sql_tool]

ROUTER_SYSTEM_PROMPT = """You are a routing assistant that decides which tool to use.

TOOLS
- PDF_RetrievalQA: For policy/rights/privacy/coverage/website policy/user guide content that lives in PDFs.
- SQL_Agent: For patients/visits/medications/prescriptions/summaries/counts/demographics in the SQLite database.

CRITICAL:
- ALWAYS pass the user's ORIGINAL NATURAL-LANGUAGE question to the chosen tool.
- NEVER translate the question into SQL yourself.
- NEVER pass SQL text as tool input. The SQL_Agent generates (or executes) SQL internally if needed.

OUTPUT:
Return the chosen tool's raw output only. It already includes:
- Source: <PDF or Database>
- Tool Used: <name>
- Answer: <final answer>
"""

router_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = initialize_agent(
    tools=tools,
    llm=router_llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    system_message=ROUTER_SYSTEM_PROMPT,
)

# -------------------------------
# 5) CLI loop
# -------------------------------
if __name__ == "__main__":
    print("🤖 Unified Agent ready. Ask about PDFs or the Database. Type 'exit' to quit.")
    while True:
        q = input("\nYou: ")
        if q.strip().lower() in {"exit", "quit"}:
            break
        try:
            out = agent.run(q)
            print("\n" + out)
        except Exception as e:
            print(f"\nError: {e}")


🤖 Unified Agent ready. Ask about PDFs or the Database. Type 'exit' to quit.


> Entering new AgentExecutor chain...

Invoking: `PDF_RetrievalQA` with `How to log in to My UNC Chart?`


Source: PDF • Files: User Guide.pdf (p.4), User Guide.pdf (p.3)
Tool Used: PDF_RetrievalQA
Answer: To log in to My UNC Chart, you need to follow these steps:
1. Go to https://myuncchart.org/My UNC Chart in your web browser.
2. Enter your My UNC Chart username and password.
3. Click on "Sign In."

If you forget your username or password, you can click on the "Forgot Username?" or "Forgot Password?" link below the login fields for assistance. You will be prompted to provide your first and last name, date of birth, and zip code to verify your identity and recover your username or password. If your email address is on file, your username will be sent to that email. If you can't recover your username or password online, you can contact the UNC Health Outpatient Access Center at (888) 996-2767 for assistance.T

In [4]:
import pkg_resources

print("📦 All installed packages in this environment:\n")
for dist in sorted(pkg_resources.working_set, key=lambda d: d.project_name.lower()):
    print(f"{dist.project_name}=={dist.version}")


📦 All installed packages in this environment:

aiohappyeyeballs==2.6.1
aiohttp==3.12.15
aiosignal==1.4.0
annotated-types==0.7.0
anyio==4.10.0
appnope==0.1.4
asttokens==3.0.0
async-timeout==4.0.3
attrs==25.3.0
certifi==2025.8.3
charset-normalizer==3.4.2
click==8.1.8
comm==0.2.3
dataclasses-json==0.6.7
debugpy==1.8.16
decorator==5.2.1
distro==1.9.0
exceptiongroup==1.3.0
executing==2.2.0
faiss-cpu==1.11.0.post1
fastapi==0.116.1
filelock==3.18.0
frozenlist==1.7.0
fsspec==2025.7.0
h11==0.16.0
hf-xet==1.1.7
httpcore==1.0.9
httpx==0.28.1
huggingface-hub==0.34.3
idna==3.10
importlib-metadata==8.7.0
ipykernel==6.30.1
ipython==8.18.1
jedi==0.19.2
jinja2==3.1.6
jiter==0.10.0
joblib==1.5.1
jsonpatch==1.33
jsonpointer==3.0.0
jupyter-client==8.6.3
jupyter-core==5.8.1
langchain==0.1.16
langchain-community==0.0.36
langchain-core==0.1.53
langchain-openai==0.1.6
langchain-text-splitters==0.0.2
langsmith==0.1.147
MarkupSafe==3.0.2
marshmallow==3.26.1
matplotlib-inline==0.1.7
mpmath==1.3.0
multidict==6.6.